# Imports

In [1]:
from numpy.random import seed
seed(888)
from tensorflow import set_random_seed
set_random_seed(404)

In [309]:
import os
import numpy as np
import tensorflow as tf

from time import strftime
from PIL import Image

# Constants

In [73]:
X_TRAIN_PATH='MNIST/digit_xtrain.csv'
X_TEST_PATH='MNIST/digit_xtest.csv'
Y_TRAIN_PATH='MNIST/digit_ytrain.csv'
Y_TEST_PATH='MNIST/digit_ytest.csv'

LOGGING_PATH='tensorboard_digit_logs/'

NR_CLASSES=10
VALIDATION_SIZE=10000
IMAGE_WIDTH=28
IMAGE_HEIGHT=28
CHANNELS=1
TOTAL_INPUTS=784

# Get the Data

In [4]:
%%time

y_train_all=np.loadtxt(Y_TRAIN_PATH,delimiter=',',dtype=int)

Wall time: 184 ms


In [5]:
y_train_all.shape

(60000,)

In [6]:
%%time

y_test_all=np.loadtxt(Y_TEST_PATH,delimiter=',',dtype=int)

Wall time: 29 ms


In [7]:
%%time

x_train_all=np.loadtxt(X_TRAIN_PATH,delimiter=',',dtype=int)

Wall time: 24.6 s


In [8]:
%%time

x_test_all=np.loadtxt(X_TEST_PATH,delimiter=',',dtype=int)

Wall time: 4.26 s


# Explore

In [9]:
x_train_all.shape

(60000, 784)

In [10]:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [11]:
y_train_all.shape

(60000,)

In [12]:
x_test_all.shape

(10000, 784)

In [13]:
y_train_all[:5]

array([5, 0, 4, 1, 9])

# Pre-Processing

In [14]:
# Re-Scale
x_train_all,x_test_all=x_train_all/255.0,x_test_all/255.0

In [15]:
x_train_all[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [16]:
values=y_train_all[:5]
np.eye(10)[values]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [17]:
np.eye(5)[[4,2]]

array([[0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.]])

In [18]:
y_train_all=np.eye(10)[y_train_all]

In [19]:
y_train_all.shape

(60000, 10)

In [20]:
y_test_all=np.eye(10)[y_test_all]

In [21]:
y_test_all.shape

(10000, 10)

In [22]:
x_val=x_train_all[:VALIDATION_SIZE]
y_val=y_train_all[:VALIDATION_SIZE]

x_train=x_train_all[VALIDATION_SIZE:]
y_train=y_train_all[VALIDATION_SIZE:]

In [23]:
x_train.shape

(50000, 784)

In [24]:
x_val.shape

(10000, 784)

# Setup Tensorflow Graph

In [429]:
X=tf.placeholder(tf.float32,shape=[None,784])
Y=tf.placeholder(tf.float32,shape=[None,NR_CLASSES])

### Architecture

In [430]:
nr_epochs=50
learning_rate=0.001

n_hidden1=512
n_hidden2=64


In [431]:
initial_w1 = tf.truncated_normal(shape=[784,n_hidden1],
                                stddev=0.1,seed=42)

In [432]:
w1=tf.Variable(initial_value=initial_w1)

In [433]:
initial_b1 = tf.constant(value=0.0,shape=[n_hidden1])
b1=tf.Variable(initial_value=initial_b1)

In [434]:
layer1_in=tf.matmul(X,w1) + b1

In [435]:
layer1_out = tf.nn.relu(layer1_in)

In [436]:
initial_w2=tf.truncated_normal(shape=[n_hidden1,n_hidden2],
                              stddev=0.1,seed=42)
w2=tf.Variable(initial_value=initial_w2)

In [437]:
initial_b2=tf.constant(value=0.0,shape=[n_hidden2])
b2=tf.Variable(initial_value=initial_b2)

In [438]:
layer2_in=tf.matmul(layer1_out,w2)+b2

In [439]:
layer2_out=tf.nn.relu(layer2_in)

In [440]:
with tf.name_scope('output_layer'):
    initial_w3=tf.truncated_normal(shape=[n_hidden2,10],
                                  stddev=0.1,seed=42)
    w3=tf.Variable(initial_value=initial_w3)

    initial_b3=tf.constant(value=0.0,shape=[10])
    b3=tf.Variable(initial_value=initial_b3)

    layer3_in=tf.matmul(layer2_out,w3)+b3
    output=tf.nn.softmax(layer3_in)

# Tensorboard Setup

In [441]:
folder_name=f'model_1 at {strftime("%H %M")}'
dir_paths=os.path.join(LOGGING_PATH,folder_name)

try:
    os.makedirs(dir_paths)
except OSError as err:
    print(err.strerror)
else:
    print('Successfully created directory')

Successfully created directory


# Loss,Optimization and Metrics

In [442]:
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y,
                                                logits=output))

In [443]:
optimizer=tf.train.AdamOptimizer(learning_rate)
train_step=optimizer.minimize(loss)

In [444]:
correct_pred=tf.equal(tf.argmax(output,axis=1),tf.argmax(Y,axis=1))
accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [445]:
tf.summary.scalar('accuracy',accuracy)

<tf.Tensor 'accuracy:0' shape=() dtype=string>

In [446]:
tf.summary.scalar('loss',loss)

<tf.Tensor 'loss:0' shape=() dtype=string>

In [447]:
x_image = tf.reshape(X,[-1,28,28,1])
tf.summary.image('image_input',x_image,max_outputs=4)

<tf.Tensor 'image_input:0' shape=() dtype=string>

# Run Session

In [448]:
sess=tf.Session()

In [449]:
merged_summary=tf.summary.merge_all()

train_writer=tf.summary.FileWriter(dir_paths+'/train')
train_writer.add_graph(sess.graph)

validation_writer=tf.summary.FileWriter(dir_paths+'/validation')

In [450]:
init=tf.global_variables_initializer()
sess.run(init)

In [451]:
w1.eval(sess)

array([[-0.02807751, -0.01377521, -0.06763297, ..., -0.02663724,
         0.02861341, -0.05550233],
       [-0.16205125, -0.18859725, -0.03102448, ..., -0.0820701 ,
        -0.03345905, -0.02453214],
       [ 0.12582639, -0.16444902,  0.13603579, ..., -0.09897225,
        -0.09923435,  0.1451435 ],
       ...,
       [-0.04471838, -0.09593774, -0.08978765, ...,  0.04240045,
        -0.18997248,  0.00134785],
       [ 0.03215451,  0.04336654, -0.18240118, ...,  0.08296242,
        -0.10039439, -0.12682591],
       [ 0.08766606, -0.15083945,  0.08048793, ...,  0.07548849,
        -0.04359084, -0.11031353]], dtype=float32)

In [452]:
b3.eval(sess)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

## Batching the Data

In [453]:
size_of_batch=1000

In [454]:
num_examples=y_train.shape[0]
nr_iterations=int(num_examples/size_of_batch)

index_in_epoch=0

In [455]:
def next_batch(batch_size,data,labels):
    
    global num_examples
    global index_in_epoch
    
    start=index_in_epoch
    index_in_epoch = index_in_epoch + batch_size
    
    if index_in_epoch>num_examples:
        start=0
        index_in_epoch=batch_size
    
    end=index_in_epoch
    
    return data[start:end],labels[start:end]

# Training Loop

In [456]:
for epoch in range(nr_epochs):
    for i in range(nr_iterations):
        
        batch_x,batch_y=next_batch(size_of_batch,x_train,y_train)
        
        feed_dictionary={X:batch_x,Y:batch_y}
        
        sess.run(train_step,feed_dict=feed_dictionary)
        
    s,batch_accuracy=sess.run(fetches=[merged_summary,accuracy],
                            feed_dict=feed_dictionary)
    
    train_writer.add_summary(s,epoch)
    
    print(f'Epoch {epoch} \t| Training Accuracy = {batch_accuracy}')
    
    
    summary,acc=sess.run(fetches=[merged_summary,accuracy],feed_dict={X:x_val,Y:y_val})
    validation_writer.add_summary(summary,epoch)
    print('########### Validation acc: ',acc)
    
print('Done Training!')

Epoch 0 	| Training Accuracy = 0.859000027179718
########### Validation acc:  0.8266
Epoch 1 	| Training Accuracy = 0.8669999837875366
########### Validation acc:  0.8454
Epoch 2 	| Training Accuracy = 0.8709999918937683
########### Validation acc:  0.8544
Epoch 3 	| Training Accuracy = 0.875
########### Validation acc:  0.861
Epoch 4 	| Training Accuracy = 0.8769999742507935
########### Validation acc:  0.8628
Epoch 5 	| Training Accuracy = 0.9679999947547913
########### Validation acc:  0.9333
Epoch 6 	| Training Accuracy = 0.9819999933242798
########### Validation acc:  0.9559
Epoch 7 	| Training Accuracy = 0.9860000014305115
########### Validation acc:  0.9613
Epoch 8 	| Training Accuracy = 0.9850000143051147
########### Validation acc:  0.9634
Epoch 9 	| Training Accuracy = 0.9860000014305115
########### Validation acc:  0.9657
Epoch 10 	| Training Accuracy = 0.9869999885559082
########### Validation acc:  0.9651
Epoch 11 	| Training Accuracy = 0.9869999885559082
########### Valid

# Make a Prediction

In [457]:
img=Image.open('MNIST/test_img.png')
img

In [458]:
bw=img.convert(mode='L')

In [459]:
img_array=np.invert(bw)

In [460]:
img_array.shape

(28, 28)

In [461]:
test_img = img_array.ravel()

In [462]:
test_img.shape

(784,)

In [463]:
prediction=sess.run(fetches=tf.argmax(output,axis=1),feed_dict={X:[test_img]})
prediction[0]

2

# Testing and Evaluation

In [464]:
test_accuracy=sess.run(fetches=accuracy,feed_dict={X:x_test_all,Y:y_test_all})
print(test_accuracy)

0.9786


In [465]:
train_writer.close()
validation_writer.close()
sess.close()
tf.reset_default_graph()